In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.4MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score,KFold

from hyperopt import hp,fmin,tpe,STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [5]:
df = pd.read_hdf('data/car.h5')

df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = "__cat"
for feat in df.columns:
  if isinstance(df[feat][0],list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] =  factorized_values
  else:
    df[feat + SUFFIX_CAT] =  factorized_values

In [7]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df['price_value'].values 

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df['price_value'].values 

  scores = cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)



DecisonTree

In [0]:
run_model(DecisionTreeRegressor(max_depth=5),cat_feats) 

(-19695.13091100928, 148.72570644015937)

Random Forest


In [0]:
model = RandomForestRegressor(max_depth=5,n_estimators=50,random_state=0)

run_model(model,cat_feats)

(-18718.657185256638, 64.5424578125788)

In [10]:
 xgb_params ={
     'max_depth': 5,
     'n_estimators':50,
     'learnig_rate':0.1,
     'seed':0
 }


 run_model(xgb.XGBRegressor(**xgb_params),cat_feats)

[06:31:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:31:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:32:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [0]:
 m = xgb.XGBRegressor(max_depth=5,n_estimators=50,learning_rate=0.1,seed=0)
 m.fit(X,y)

 imp = PermutationImportance(m,random_state=0).fit(X,y)
 eli5.show_weights(imp,feature_names = cat_feats)

[09:38:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
feats =['param_napęd__cat',
'param_rok-produkcji__cat',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

run_model(xgb.XGBRegressor(**xgb_params),feats)

[09:46:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:46:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:46:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x:-1 if str(x) == 'None' else int(x) )

In [0]:
feats =['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

run_model(xgb.XGBRegressor(**xgb_params),feats)

[09:54:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:54:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:55:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [18]:
df['param_moc'] = df['param_moc'].map(lambda x:-1 if str(x) == 'None' else int(x.split(' ')[0]))

AttributeError: ignored

In [19]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x:-1 if str(x) == 'None' else int(x) )

#df['param_moc'] = df['param_moc'].map(lambda x:-1 if str(x) == 'None' else int(x.split(' ')[0]))

feats =['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

run_model(xgb.XGBRegressor(**xgb_params),feats)

[06:51:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:51:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:51:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [0]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [21]:
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [22]:
feats =['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

run_model(xgb.XGBRegressor(**xgb_params),feats)

[06:52:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:52:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:52:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [0]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  
  mean_mae, score_std = run_model(xgb.XGBRFRegressor(**params), feats)

  return{'loss' : np.abs(mean_mae), 'status': STATUS_OK}

  #space
xgb_reg_params = {
    'learning_rate':      hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth':          hp.choice('max_depth', np.arange(5,16,1, dtype = int)),
    'subsample':          hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bythree':  hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':          'reg:squarederror',
    'n_estimators':       100,
    'seed':               0,
  }

In [31]:
#run
best = fmin(obj_func,xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bythree': 0.75, 'learning_rate': 0.25, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bythree': 0.8500000000000001, 'learning_rate': 0.2, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bythree': 0.5, 'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bythree': 0.65, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bythree': 0.8, 'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bythree': 0.7000000000000001, 'learning_rate': 0.25, 'ma

{'colsample_bytree': 0.55,
 'learning_rate': 5,
 'max_depth': 4,
 'subsample': 1.0}